In [1]:
import os
# Set the working directory to your project's main folder
os.chdir('/mnt/c/Users/pablosal/Desktop/lilly-workshop-gbb-text-to-speach/')
import azure.cognitiveservices.speech as speechsdk
from utils.ml_logging import get_logger
from dotenv import load_dotenv
load_dotenv()
logger = get_logger()

In [2]:
KEY = os.getenv('KEY')
REGION = os.getenv('REGION')
FILE_NAME = 'notebooks/dev/8000khz-mulaw-pullstream/7.wav'

In [6]:
def from_file(file_name: str, key: str, region: str) -> str:
    """
    Transcribes speech from an audio file using Azure Cognitive Services Speech SDK.

    Args:
        file_name (str): The name of the audio file to transcribe.
        key (str): The subscription key for the Speech service.
        region (str): The region for the Speech service.

    Returns:
        str: The transcribed text from the audio file.
    """
    speech_config = speechsdk.SpeechConfig(subscription=key, region=region)
    audio_config = speechsdk.AudioConfig(filename=file_name)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    logger.info(f"Transcribing speech from file: {file_name}")
    result = speech_recognizer.recognize_once_async().get()
    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        logger.info(f"Transcription result: {result.text}")
    elif result.reason == speechsdk.ResultReason.NoMatch:
        logger.warning(f"No speech could be recognized: {result.no_match_details}")
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        logger.error(f"Speech Recognition canceled: {cancellation_details.reason}")
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            logger.error(f"Error details: {cancellation_details.error_details}")
    return result.text


In [10]:
from_file(file_name="/mnt/c/Users/pablosal/Desktop/lilly-workshop-gbb-text-to-speach/notebooks/dev/8000khz-mulaw-pullstream/7.wav", key=KEY, region=REGION)

2023-11-14 16:20:10,238 - micro - MainProcess - INFO     Transcribing speech from file: /mnt/c/Users/pablosal/Desktop/lilly-workshop-gbb-text-to-speach/notebooks/dev/8000khz-mulaw-pullstream/7.wav (4106455139.py:from_file:17)
2023-11-14 16:20:10,244 - micro - MainProcess - ERROR    Speech Recognition canceled: CancellationReason.Error (4106455139.py:from_file:25)
2023-11-14 16:20:10,245 - micro - MainProcess - ERROR    Error details: Runtime error: Failed to initialize platform (azure-c-shared). Error: 2153 SessionId: b498113da2c34b4bb977f0e956982074 (4106455139.py:from_file:27)


''

In [5]:
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license. See LICENSE.md file in the project root for full license information.

# <code>
import azure.cognitiveservices.speech as speechsdk

# Creates an instance of a speech config with specified subscription key and service region.
# Replace with your own subscription key and service region (e.g., "westus").
speech_config = speechsdk.SpeechConfig(subscription=KEY, region=REGION)

# Creates a recognizer with the given settings
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config)

print("Say something...")



RuntimeError: Exception with error code: 
[CALL STACK BEGIN]

/home/pablosal/miniconda3/envs/lilly-speach-to-text/lib/python3.9/site-packages/azure/cognitiveservices/speech/libMicrosoft.CognitiveServices.Speech.extension.audio.sys.so(+0xe1e9) [0x7f761f40e1e9]
/home/pablosal/miniconda3/envs/lilly-speach-to-text/lib/python3.9/site-packages/azure/cognitiveservices/speech/libMicrosoft.CognitiveServices.Speech.core.so(+0x1f44f9) [0x7f761f9f44f9]
/home/pablosal/miniconda3/envs/lilly-speach-to-text/lib/python3.9/site-packages/azure/cognitiveservices/speech/libMicrosoft.CognitiveServices.Speech.core.so(+0xff4f7) [0x7f761f8ff4f7]
/home/pablosal/miniconda3/envs/lilly-speach-to-text/lib/python3.9/site-packages/azure/cognitiveservices/speech/libMicrosoft.CognitiveServices.Speech.core.so(+0x1af42f) [0x7f761f9af42f]
/home/pablosal/miniconda3/envs/lilly-speach-to-text/lib/python3.9/site-packages/azure/cognitiveservices/speech/libMicrosoft.CognitiveServices.Speech.core.so(+0xf108d) [0x7f761f8f108d]
/h

In [ ]:
# Starts speech recognition, and returns after a single utterance is recognized. The end of a
# single utterance is determined by listening for silence at the end or until a maximum of 15
# seconds of audio is processed.  The task returns the recognition text as result. 
# Note: Since recognize_once() returns only a single utterance, it is suitable only for single
# shot recognition like command or query. 
# For long-running multi-utterance recognition, use start_continuous_recognition() instead.
result = speech_recognizer.recognize_once()

# Checks result.
if result.reason == speechsdk.ResultReason.RecognizedSpeech:
    print("Recognized: {}".format(result.text))
elif result.reason == speechsdk.ResultReason.NoMatch:
    print("No speech could be recognized: {}".format(result.no_match_details))
elif result.reason == speechsdk.ResultReason.Canceled:
    cancellation_details = result.cancellation_details
    print("Speech Recognition canceled: {}".format(cancellation_details.reason))
    if cancellation_details.reason == speechsdk.CancellationReason.Error:
        print("Error details: {}".format(cancellation_details.error_details))

In [ ]:
speech_config = speechsdk.SpeechConfig(subscription=KEY, region=REGION)

In [ ]:
speech_config.enable_audio_logging()

In [ ]:
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config)

In [ ]:
audio_config = speechsdk.AudioConfig(filename=FILE_NAME)

In [ ]:
audio_config.get_property?

In [ ]:
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

In [ ]:
speech_recognizer

In [ ]:
result = speech_recognizer.recognize_once_async().get()

In [ ]:
result.text

In [ ]:
speech_config

In [ ]:
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config)